# Stock Trend Prediction - Model Training

- Creates sequences from time series features
- Builds and compare neural network architectures (LSTM, GRU)
- Trains models using KhayatMiniNN
- Compares model performance
- Selects best model for prediction


In [22]:
import sys
import os
import pandas as pd
import numpy as np
from pathlib import Path
import pickle
import importlib

sys.path.insert(0, os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))

# Reload KhayatMiniNN modules to pick up any code changes
import KhayatMiniNN.optimizers.adam as adam_module
importlib.reload(adam_module)

from KhayatMiniNN.neural_network import NeuralNetwork
from KhayatMiniNN.trainer import Trainer
from KhayatMiniNN.layers import LSTM, GRU, Conv1D, Dense, ReLU, Sigmoid, MaxPooling1D
from KhayatMiniNN.layers.base import Layer
from KhayatMiniNN.regularization import Dropout, BatchNormalization
from KhayatMiniNN.losses import BinaryCrossEntropy
from KhayatMiniNN.optimizers.adam import Adam  # Import directly from reloaded module
from KhayatMiniNN.utils.sequence_utils import create_sequences

import warnings
warnings.filterwarnings('ignore')

# Weighted BinaryCrossEntropy wrapper for class imbalance - IMPROVED VERSION
class WeightedBinaryCrossEntropy:
    """
    Improved weighted BinaryCrossEntropy with per-sample weighting and optional focal loss.
    Applies different weights to positive and negative classes with per-sample gradient scaling.
    """
    def __init__(self, weight_for_0, weight_for_1, from_logits=False, focal_gamma=0.0):
        """
        Args:
            weight_for_0: Weight for class 0 (negative/Lower)
            weight_for_1: Weight for class 1 (positive/Higher)
            from_logits: Whether predictions are logits or probabilities
            focal_gamma: Focal loss gamma parameter (0 = no focal loss, 2.0 = strong focus on hard examples)
        """
        self.base_loss = BinaryCrossEntropy(from_logits=from_logits)
        self.weight_for_0 = weight_for_0
        self.weight_for_1 = weight_for_1
        self.from_logits = from_logits
        self.focal_gamma = focal_gamma
        self.predictions = None
        self.targets = None
        self.weights = None
    
    def forward(self, predictions, targets):
        """Compute weighted binary cross-entropy loss with optional focal loss."""
        self.predictions = predictions
        self.targets = targets
        
        # Ensure correct shapes
        if targets.ndim == 1:
            targets_reshaped = targets.reshape(-1, 1)
        else:
            targets_reshaped = targets
        
        if predictions.ndim == 1:
            predictions_reshaped = predictions.reshape(-1, 1)
        else:
            predictions_reshaped = predictions
        
        # Clip predictions to avoid log(0)
        eps = 1e-7
        predictions_clipped = np.clip(predictions_reshaped, eps, 1 - eps)
        
        # Calculate per-sample BCE loss
        bce = -(targets_reshaped * np.log(predictions_clipped) + 
                (1 - targets_reshaped) * np.log(1 - predictions_clipped))
        
        # Calculate per-sample weights
        self.weights = np.where(targets_reshaped == 0, self.weight_for_0, self.weight_for_1)
        
        # Apply focal loss modulation if gamma > 0
        if self.focal_gamma > 0:
            # pt = p if y=1, else (1-p)
            pt = np.where(targets_reshaped == 1, predictions_clipped, 1 - predictions_clipped)
            focal_weight = (1 - pt) ** self.focal_gamma
            bce = focal_weight * bce
        
        # Apply class weights per sample
        weighted_bce = self.weights * bce
        
        return np.mean(weighted_bce)
    
    def backward(self):
        """Return weighted gradient."""
        eps = 1e-7
        predictions = self.predictions
        targets = self.targets
        
        if targets.ndim == 1:
            targets = targets.reshape(-1, 1)
        if predictions.ndim == 1:
            predictions = predictions.reshape(-1, 1)
        
        predictions_clipped = np.clip(predictions, eps, 1 - eps)
        
        # Base gradient: (p - y) / (p * (1 - p))
        grad = (predictions_clipped - targets) / (predictions_clipped * (1 - predictions_clipped) + eps)
        
        # Apply class weights to gradients
        weighted_grad = self.weights * grad / targets.shape[0]
        
        return weighted_grad
    
    def set_loss(self, loss_fn):
        """Compatibility method - allows setting base loss if needed."""
        pass

class Flatten(Layer):
    """Flatten layer to convert 3D (batch, seq, features) to 2D (batch, seq*features)."""
    def __init__(self, name="Flatten"):
        super().__init__(name)
    
    def forward(self, input_data):
        self.input = input_data
        batch_size = input_data.shape[0]
        return input_data.reshape(batch_size, -1)
    
    def backward(self, output_grad):
        return output_grad.reshape(self.input.shape)


## 1. Load Feature-Engineered Data


In [23]:
data_dir = Path("../data/processed")
train_df = pd.read_csv(data_dir / "train_features.csv")
val_df = pd.read_csv(data_dir / "val_features.csv")

with open(data_dir / "feature_cols.pkl", "rb") as f:
    feature_cols = pickle.load(f)

# Ensure the CSV has all columns listed in feature_cols (avoids KeyError later)
missing = [c for c in feature_cols if c not in train_df.columns]
if missing:
    raise KeyError(
        "Some features in feature_cols.pkl are missing from train_features.csv, e.g.: "
        + str(missing[:10])
        + ". Re-run the feature engineering notebook (2_feature_engineering) to regenerate the processed files."
    )

print(f"Train: {len(train_df):,} samples. Val: {len(val_df):,}. Features: {len(feature_cols)}.")


Train: 5,505,350 samples. Val: 5,475,700. Features: 71.


## 2. Create Sequences

We'll create sequences from the time series data. Each sequence will have a fixed length (e.g., 30 days) and predict the target.


In [24]:
# Training Parameters - TUNED FOR BETTER RESULTS
SEQUENCE_LENGTH = 20  # Reduced from 30 to 20 - shorter sequences reduce noise and overfitting
BATCH_SIZE = 64  # Increased from 16 to 64 - larger batches provide more stable gradients
EPOCHS = 30  # Increased from 20 to 30 - allow more training time with lower LR
LEARNING_RATE = 0.0002  # Reduced from 0.0005 for smoother convergence and less overfitting
EARLY_STOPPING_PATIENCE = 5  # Reduced to 5 to stop before overfitting kicks in

# Memory management: Limit dataset size to avoid memory errors and disk space issues
# 100000 samples ≈ 1.3 GB, 150000 samples ≈ 1.9 GB, 200000 samples ≈ 2.6 GB
# Further reduced to prevent macOS swap file creation (which consumes disk space)
MAX_SAMPLES = 150000  # Increased to 150k for better generalization (more data helps reduce overfitting)
SAMPLE_RATE = 1.0  # Sample rate for sequences (1.0 = all, 0.1 = 10% of sequences)

# Set random seed for reproducibility
np.random.seed(42)

# In-memory sequence creation (no disk storage)
def create_sequences_in_memory(df, feature_cols, sequence_length, target_col='target', 
                               max_samples=None, sample_rate=1.0):
    """
    Create sequences in memory grouped by ticker.
    Optionally limit dataset size with max_samples or sample_rate.
    Uses pre-allocated arrays when max_samples is set for better memory efficiency.
    """
    if max_samples is None:
        raise ValueError("max_samples must be set to avoid memory errors. Try MAX_SAMPLES = 500000 or adjust based on available RAM.")
    
    # Pre-allocate arrays for better memory management
    X = np.zeros((max_samples, sequence_length, len(feature_cols)), dtype=np.float32)
    y = np.zeros(max_samples, dtype=np.float32)
    current_idx = 0
    
    print("  Creating sequences per ticker...")
    tickers = df['Ticker'].unique()
    
    for ticker_idx, ticker in enumerate(tickers):
        if current_idx >= max_samples:
            break
            
        ticker_data = df[df['Ticker'] == ticker].sort_values('Date')
        features = ticker_data[feature_cols].values.astype(np.float32)
        targets = ticker_data[target_col].values.astype(np.float32)
        
        if len(features) < sequence_length:
            continue
        
        # Create sliding window sequences for this ticker
        num_sequences = len(features) - sequence_length + 1
        
        # Apply sampling if needed
        if sample_rate < 1.0:
            num_sequences = int(num_sequences * sample_rate)
            indices = np.random.choice(len(features) - sequence_length + 1, 
                                      size=num_sequences, replace=False)
            indices = np.sort(indices)
        else:
            indices = np.arange(len(features) - sequence_length + 1)
        
        for i in indices:
            if current_idx >= max_samples:
                break
            X[current_idx] = features[i:i+sequence_length]
            y[current_idx] = targets[i+sequence_length-1]
            current_idx += 1
        
        if (ticker_idx + 1) % 100 == 0:
            print(f"  Processed {ticker_idx + 1}/{len(tickers)} tickers, {current_idx:,} sequences...", end='\r')
    
    print(f"  Completed: {current_idx:,} sequences created")
    
    # Trim to actual size
    if current_idx < max_samples:
        X = X[:current_idx]
        y = y[:current_idx]
    
    return X, y

seq_dir = Path("../data/sequences")
if seq_dir.exists():
    print("Cleaning up existing sequence files to free disk space...")
    for file in seq_dir.glob("*.dat"):
        try:
            file.unlink()
            print(f"  Deleted: {file.name}")
        except Exception as e:
            print(f"  Could not delete {file.name}: {e}")

print("\n" + "=" * 60)
print("Creating Sequences")
print("=" * 60)
print(f"\nUsing MAX_SAMPLES = {MAX_SAMPLES:,} (≈{MAX_SAMPLES * 30 * 115 * 4 / 1024**3:.1f} GB)")

print("\nCreating sequences for train set...")
X_train, y_train = create_sequences_in_memory(
    train_df, feature_cols, SEQUENCE_LENGTH, 
    max_samples=MAX_SAMPLES, sample_rate=SAMPLE_RATE
)
print(f"Train sequences: {X_train.shape}")
print(f"Memory usage: {X_train.nbytes / 1024**3:.2f} GB (float32)")

print("\nCreating sequences for validation set...")
# Use same sampling for validation, but limit to reasonable size
val_max_samples = MAX_SAMPLES // 10 if MAX_SAMPLES else None  # 10% of train size
X_val, y_val = create_sequences_in_memory(
    val_df, feature_cols, SEQUENCE_LENGTH,
    max_samples=val_max_samples, sample_rate=SAMPLE_RATE
)
print(f"Val sequences: {X_val.shape}")
print(f"Memory usage: {X_val.nbytes / 1024**3:.2f} GB (float32)")

# Calculate class weights for imbalanced data
from sklearn.utils.class_weight import compute_class_weight

# Calculate class weights
classes = np.unique(y_train)
class_weights = compute_class_weight('balanced', classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))

print(f"\nClass weights (for imbalanced data):")
print(f"  Class 0 (Lower): {class_weight_dict[0]:.4f}")
print(f"  Class 1 (Higher): {class_weight_dict[1]:.4f}")

# Store for use in training
weight_for_0 = class_weight_dict[0]
weight_for_1 = class_weight_dict[1]

print(f"\nTarget distribution (train):")
print(f"(Higher): {y_train.sum():.0f} ({y_train.mean()*100:.2f}%)")
print(f"(Lower): {(len(y_train) - y_train.sum()):.0f} ({(1-y_train.mean())*100:.2f}%)")

print(f"\nTarget distribution (val):")
print(f"(Higher): {y_val.sum():.0f} ({y_val.mean()*100:.2f}%)")
print(f"(Lower): {(len(y_val) - y_val.sum()):.0f} ({(1-y_val.mean())*100:.2f}%)")

if MAX_SAMPLES:
    print(f"Dataset limited to {MAX_SAMPLES:,} samples per set.")
if SAMPLE_RATE < 1.0:
    print(f"Using {SAMPLE_RATE*100:.1f}% of available sequences.")

# Free up memory by deleting dataframes (sequences are already created)
del train_df, val_df
import gc
gc.collect()
print("\n✓ Freed memory from dataframes")



Creating Sequences

Using MAX_SAMPLES = 150,000 (≈1.9 GB)

Creating sequences for train set...
  Creating sequences per ticker...
  Completed: 150,000 sequences created
Train sequences: (150000, 20, 71)
Memory usage: 0.79 GB (float32)

Creating sequences for validation set...
  Creating sequences per ticker...
  Completed: 15,000 sequences created
Val sequences: (15000, 20, 71)
Memory usage: 0.08 GB (float32)

Class weights (for imbalanced data):
  Class 0 (Lower): 1.0985
  Class 1 (Higher): 0.9177

Target distribution (train):
(Higher): 81722 (54.48%)
(Lower): 68278 (45.52%)

Target distribution (val):
(Higher): 8657 (57.71%)
(Lower): 6343 (42.29%)
Dataset limited to 150,000 samples per set.

✓ Freed memory from dataframes


## 3. Build Multiple Model Architectures

We'll create and train multiple models:
1. **LSTM** - Long Short-Term Memory
2. **GRU** - Gated Recurrent Unit  


In [29]:
# Model parameters - TUNED FOR BETTER RESULTS
input_size = len(feature_cols)
hidden_size = 64
# Further reduced hidden sizes to prevent overfitting
lstm_hidden = 32  # Reduced from 48 to 32 for better generalization
gru_hidden = 48  # Reduced from 64 to 48 to match LSTM complexity
conv_filters = 32

# Focal loss gamma - helps focus on hard-to-classify examples (reduces bias toward majority class)
FOCAL_GAMMA = 1.0  # Reduced from 1.5 to 1.0 to prevent GRU collapse

# Class weight multiplier - boost weight for minority class to balance predictions
CLASS_WEIGHT_BOOST = 1.2  # Reduced from 1.5 to 1.2 for more stable training

# L2 regularization (weight decay) - helps prevent overfitting
L2_WEIGHT_DECAY = 0.001  # Small L2 penalty on weights

# Helper function to create and train a model
def build_and_train_model(model_name, architecture_func, X_train, y_train, X_val, y_val, 
                         use_weighted_loss=True, use_focal_loss=True, custom_lr=None):
    """Build, train, and evaluate a model with optional weighted loss for class imbalance.
    
    Args:
        use_focal_loss: If False, uses standard weighted BCE without focal modulation (helps GRU)
        custom_lr: Optional custom learning rate for this model
    """
    import gc
    import sys
    
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    # Build model
    model = architecture_func()
    model.summary()
    
    # Set loss - use weighted loss with optional focal loss
    if use_weighted_loss and 'weight_for_0' in globals() and 'weight_for_1' in globals():
        # Boost the minority class weight to balance predictions
        boosted_weight_0 = weight_for_0 * CLASS_WEIGHT_BOOST
        focal_gamma = FOCAL_GAMMA if use_focal_loss else 0.0
        loss_fn = WeightedBinaryCrossEntropy(
            weight_for_0=boosted_weight_0, 
            weight_for_1=weight_for_1, 
            from_logits=False,
            focal_gamma=focal_gamma
        )
        if use_focal_loss:
            print(f"\n✓ Using weighted focal loss (gamma={focal_gamma})")
        else:
            print(f"\n✓ Using weighted BCE (no focal loss - more stable for some architectures)")
        print(f"  Class 0 weight: {boosted_weight_0:.4f} (boosted), Class 1 weight: {weight_for_1:.4f}")
    else:
        loss_fn = BinaryCrossEntropy(from_logits=False)
        print(f"\n⚠ Using standard loss (no class weights)")
    
    model.set_loss(loss_fn)
    
    # Create optimizer and trainer with L2 regularization (weight decay)
    lr = custom_lr if custom_lr is not None else LEARNING_RATE
    optimizer = Adam(learning_rate=lr, weight_decay=L2_WEIGHT_DECAY)
    trainer = Trainer(model, optimizer, loss_fn)
    print(f"  Learning rate: {lr}")
    
    # Train with early stopping - now tracking best validation accuracy (better for classification)
    best_val_acc = 0.0
    best_val_loss = float('inf')
    patience_counter = 0
    best_params = None
    
    print(f"\nTraining for up to {EPOCHS} epochs...")
    print(f"Early stopping based on validation accuracy (patience={EARLY_STOPPING_PATIENCE})")
    
    for epoch in range(EPOCHS):
        # Aggressive memory cleanup before each epoch
        gc.collect()
        
        # Train one epoch
        history = trainer.fit(
            X_train, y_train.reshape(-1, 1),
            X_val=X_val, y_val=y_val.reshape(-1, 1),
            epochs=1,
            batch_size=BATCH_SIZE,
            verbose=False
        )
        
        # Get validation metrics
        val_loss, val_acc = trainer.evaluate(X_val, y_val.reshape(-1, 1))
        
        # Calculate class-specific metrics every few epochs
        if (epoch + 1) % 3 == 0 or epoch == 0:
            train_loss, train_acc = trainer.evaluate(X_train, y_train.reshape(-1, 1))
            
            # Get predictions for class-specific metrics
            y_val_pred = (trainer.predict(X_val) > 0.5).astype(int).flatten()
            class_0_acc = np.mean(y_val_pred[y_val == 0] == 0) * 100 if np.sum(y_val == 0) > 0 else 0
            class_1_acc = np.mean(y_val_pred[y_val == 1] == 1) * 100 if np.sum(y_val == 1) > 0 else 0
            
            print(f"Epoch {epoch+1}/{EPOCHS} | "
                  f"Train: {train_acc:.1f}% | Val: {val_acc:.1f}% | "
                  f"Gap: {train_acc - val_acc:+.1f}% | "
                  f"Class0: {class_0_acc:.1f}% | Class1: {class_1_acc:.1f}%")
        
        # Early stopping based on validation accuracy (better than loss for classification)
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_val_loss = val_loss
            patience_counter = 0
            # Store best params
            best_params = model.get_params()
        else:
            patience_counter += 1
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly stopping at epoch {epoch+1} (no improvement for {EARLY_STOPPING_PATIENCE} epochs)")
                break
        
        # Force garbage collection every epoch
        gc.collect()
    
    # Restore best parameters
    if best_params:
        model.set_params(best_params)
    
    # Final evaluation
    val_loss, val_accuracy = trainer.evaluate(X_val, y_val.reshape(-1, 1))
    train_loss, train_accuracy = trainer.evaluate(X_train, y_train.reshape(-1, 1))
    
    # Final class-specific metrics
    y_val_pred = (trainer.predict(X_val) > 0.5).astype(int).flatten()
    final_class_0_acc = np.mean(y_val_pred[y_val == 0] == 0) * 100 if np.sum(y_val == 0) > 0 else 0
    final_class_1_acc = np.mean(y_val_pred[y_val == 1] == 1) * 100 if np.sum(y_val == 1) > 0 else 0
    
    print(f"\n✓ Training completed!")
    print(f"  Best Val Accuracy: {best_val_acc:.2f}% (Loss: {best_val_loss:.4f})")
    print(f"  Final Val Accuracy: {val_accuracy:.2f}%")
    print(f"  Train Accuracy: {train_accuracy:.2f}%")
    print(f"  Class 0 (Lower) Accuracy: {final_class_0_acc:.2f}%")
    print(f"  Class 1 (Higher) Accuracy: {final_class_1_acc:.2f}%")
    
    # Clear trainer optimizer states to free memory
    if hasattr(trainer, 'layer_optimizers'):
        trainer.layer_optimizers.clear()
    
    return model, trainer, {
        'name': model_name,
        'val_loss': best_val_loss,
        'val_accuracy': best_val_acc,  # Use best accuracy instead of final
        'train_accuracy': train_accuracy,
        'class_0_accuracy': final_class_0_acc,
        'class_1_accuracy': final_class_1_acc,
        'history': history
    }

# Define model architectures - TUNED FOR BETTER RESULTS
def build_lstm_model():
    """Build optimized LSTM model with strong regularization to prevent overfitting."""
    model = NeuralNetwork(name="LSTM_Stock_Predictor_V2")
    # LSTM layer with smaller hidden size
    model.add_layer(LSTM(input_size, lstm_hidden, return_sequences=False), name="lstm1")
    # Batch normalization for stable training
    model.add_layer(BatchNormalization(lstm_hidden), name="batchnorm1")
    # Higher dropout to aggressively prevent overfitting
    model.add_layer(Dropout(dropout_rate=0.6), name="dropout1")
    # First dense layer - smaller size for simpler model
    model.add_layer(Dense(lstm_hidden, 16), name="dense1")
    model.add_layer(BatchNormalization(16), name="batchnorm2")
    model.add_layer(ReLU(), name="relu1")
    # Strong dropout before output
    model.add_layer(Dropout(dropout_rate=0.5), name="dropout2")
    # Output layer
    model.add_layer(Dense(16, 1), name="dense2")
    model.add_layer(Sigmoid(), name="sigmoid1")
    return model

def build_gru_model():
    """Build GRU model - simplified architecture to prevent collapse."""
    model = NeuralNetwork(name="GRU_Stock_Predictor_V3")
    # GRU layer - simpler, larger hidden size for better gradient flow
    model.add_layer(GRU(input_size, 64, return_sequences=False), name="gru1")
    # Lower dropout to prevent gradient starvation
    model.add_layer(Dropout(dropout_rate=0.4), name="dropout1")
    # Direct to output with one hidden layer - simpler architecture
    model.add_layer(Dense(64, 32), name="dense1")
    model.add_layer(ReLU(), name="relu1")
    model.add_layer(Dropout(dropout_rate=0.3), name="dropout2")
    model.add_layer(Dense(32, 1), name="dense2")
    model.add_layer(Sigmoid(), name="sigmoid1")
    return model

def build_conv1d_model():
    """Build Conv1D model with proper shape handling - TUNED."""
    # Calculate output shape after convolutions and pooling
    # Input: (batch, SEQUENCE_LENGTH, input_size)
    # After conv1 (same padding): (batch, seq_len, conv_filters)
    # After pool1 (pool_size=2, stride=2): (batch, seq_len//2, conv_filters)
    # After conv2 (same padding): (batch, seq_len//2, conv_filters*2)
    # After pool2 (pool_size=2, stride=2): (batch, seq_len//4, conv_filters*2)
    
    seq_after_pool1 = (SEQUENCE_LENGTH - 2) // 2 + 1
    seq_after_pool2 = (seq_after_pool1 - 2) // 2 + 1
    flattened_size = seq_after_pool2 * (conv_filters * 2)
    
    model = NeuralNetwork(name="Conv1D_Stock_Predictor_V2")
    model.add_layer(Conv1D(input_size, conv_filters, kernel_size=3, padding='same'), name="conv1")
    model.add_layer(BatchNormalization(conv_filters), name="batchnorm1")
    model.add_layer(ReLU(), name="relu1")
    model.add_layer(MaxPooling1D(pool_size=2, stride=2), name="pool1")
    model.add_layer(Dropout(dropout_rate=0.3), name="dropout1")
    model.add_layer(Conv1D(conv_filters, conv_filters*2, kernel_size=3, padding='same'), name="conv2")
    model.add_layer(BatchNormalization(conv_filters*2), name="batchnorm2")
    model.add_layer(ReLU(), name="relu2")
    model.add_layer(MaxPooling1D(pool_size=2, stride=2), name="pool2")
    model.add_layer(Flatten(), name="flatten")
    model.add_layer(Dense(flattened_size, 32), name="dense1")
    model.add_layer(BatchNormalization(32), name="batchnorm3")
    model.add_layer(ReLU(), name="relu3")
    model.add_layer(Dropout(dropout_rate=0.5), name="dropout2")
    model.add_layer(Dense(32, 1), name="dense2")
    model.add_layer(Sigmoid(), name="sigmoid1")
    return model

def build_hybrid_model():
    """Build Conv1D + LSTM hybrid model - TUNED."""
    model = NeuralNetwork(name="Hybrid_ConvLSTM_V2")
    # Conv1D layers for feature extraction
    model.add_layer(Conv1D(input_size, conv_filters, kernel_size=3, padding='same'), name="conv1")
    model.add_layer(BatchNormalization(conv_filters), name="batchnorm1")
    model.add_layer(ReLU(), name="relu1")
    model.add_layer(MaxPooling1D(pool_size=2, stride=2), name="pool1")
    model.add_layer(Dropout(dropout_rate=0.3), name="dropout1")
    # LSTM for sequence modeling
    model.add_layer(LSTM(conv_filters, lstm_hidden, return_sequences=False), name="lstm1")
    model.add_layer(BatchNormalization(lstm_hidden), name="batchnorm2")
    model.add_layer(Dropout(dropout_rate=0.6), name="dropout2")
    model.add_layer(Dense(lstm_hidden, 16), name="dense1")
    model.add_layer(ReLU(), name="relu2")
    model.add_layer(Dropout(dropout_rate=0.5), name="dropout3")
    model.add_layer(Dense(16, 1), name="dense2")
    model.add_layer(Sigmoid(), name="sigmoid1")
    return model

print("\nModel architectures defined. Ready to train!")



Model architectures defined. Ready to train!


## 4. Train All Models

We'll train all model architectures and compare their performance.


In [30]:
# Train all models - Ultra Memory-optimized version
# Only keep results and best model, not all models in memory
import gc
import sys

models_results = {}
trained_models = {}
best_model_name = None
best_model = None
best_trainer = None
best_val_acc = 0.0

# Train LSTM (with focal loss)
print("\n" + "="*60)
print("Training LSTM...")
print("="*60)
lstm_model, lstm_trainer, lstm_results = build_and_train_model(
    "LSTM", build_lstm_model, X_train, y_train, X_val, y_val,
    use_focal_loss=True
)
models_results['LSTM'] = lstm_results
if lstm_results['val_accuracy'] > best_val_acc:
    best_val_acc = lstm_results['val_accuracy']
    best_model_name = 'LSTM'
    best_model = lstm_model
    best_trainer = lstm_trainer
trained_models['LSTM'] = (lstm_model, lstm_trainer)
del lstm_model, lstm_trainer
gc.collect()
print("✓ Memory freed after LSTM training")

# Train GRU (WITHOUT focal loss - prevents collapse, with higher LR)
print("\n" + "="*60)
print("Training GRU (no focal loss for stability)...")
print("="*60)
gc.collect()
gru_model, gru_trainer, gru_results = build_and_train_model(
    "GRU", build_gru_model, X_train, y_train, X_val, y_val,
    use_focal_loss=False,  # Disable focal loss to prevent collapse
    custom_lr=0.0005  # Higher LR for GRU to escape local minima
)
models_results['GRU'] = gru_results
if gru_results['val_accuracy'] > best_val_acc:
    best_val_acc = gru_results['val_accuracy']
    best_model_name = 'GRU'
    best_model = gru_model
    best_trainer = gru_trainer
trained_models['GRU'] = (gru_model, gru_trainer)
del gru_model, gru_trainer
gc.collect()
print("✓ Memory freed after GRU training")

# Train Conv1D
print("\n" + "="*60)
print("Training Conv1D...")
print("="*60)
gc.collect()
conv_model, conv_trainer, conv_results = build_and_train_model(
    "Conv1D", build_conv1d_model, X_train, y_train, X_val, y_val,
    use_focal_loss=True
)
models_results['Conv1D'] = conv_results
if conv_results['val_accuracy'] > best_val_acc:
    best_val_acc = conv_results['val_accuracy']
    best_model_name = 'Conv1D'
    best_model = conv_model
    best_trainer = conv_trainer
trained_models['Conv1D'] = (conv_model, conv_trainer)
del conv_model, conv_trainer
gc.collect()
print("✓ Memory freed after Conv1D training")

# Train Hybrid (Conv1D + LSTM)
print("\n" + "="*60)
print("Training Hybrid (Conv1D + LSTM)...")
print("="*60)
gc.collect()
hybrid_model, hybrid_trainer, hybrid_results = build_and_train_model(
    "Hybrid", build_hybrid_model, X_train, y_train, X_val, y_val,
    use_focal_loss=True
)
models_results['Hybrid'] = hybrid_results
if hybrid_results['val_accuracy'] > best_val_acc:
    best_val_acc = hybrid_results['val_accuracy']
    best_model_name = 'Hybrid'
    best_model = hybrid_model
    best_trainer = hybrid_trainer
trained_models['Hybrid'] = (hybrid_model, hybrid_trainer)
del hybrid_model, hybrid_trainer
gc.collect()
print("✓ Memory freed after Hybrid training")

print(f"\n{'='*60}")
print("All Models Trained!")
print(f"{'='*60}")
print(f"🏆 Best Model: {best_model_name} (Val Accuracy: {best_val_acc:.2f}%)")


Training LSTM...

Training LSTM

Network: LSTM_Stock_Predictor_V2
Layer Name           Type                 Parameters          
------------------------------------------------------------
lstm1                LSTM                 13312               
batchnorm1           BatchNormalization   64                  
dropout1             Dropout              0                   
dense1               Dense                528                 
batchnorm2           BatchNormalization   32                  
relu1                ReLU                 0                   
dropout2             Dropout              0                   
dense2               Dense                17                  
sigmoid1             Sigmoid              0                   
------------------------------------------------------------
Total parameters: 13953


✓ Using weighted focal loss (gamma=1.0)
  Class 0 weight: 1.3181 (boosted), Class 1 weight: 0.9177
  Learning rate: 0.0002

Training for up to 30 epochs...

## 5. Compare Models and Select Best


In [31]:
# Compare all models - Select best by validation accuracy (better for classification)
print("\n" + "="*60)
print("Model Comparison")
print("="*60)

comparison_df = pd.DataFrame([
    {
        'Model': name,
        'Val Loss': results['val_loss'],
        'Val Accuracy': results['val_accuracy'],
        'Train Accuracy': results['train_accuracy'],
        'Class 0 Acc': results.get('class_0_accuracy', 0),
        'Class 1 Acc': results.get('class_1_accuracy', 0)
    }
    for name, results in models_results.items()
]).sort_values('Val Accuracy', ascending=False)  # Sort by accuracy (descending)

# Calculate balance score (how well balanced are predictions between classes)
comparison_df['Balance'] = 100 - abs(comparison_df['Class 0 Acc'] - comparison_df['Class 1 Acc'])

print("\n" + comparison_df.to_string(index=False))

# Select best model by validation accuracy (more meaningful for classification)
# Override the loss-based selection with accuracy-based selection
best_model_name_by_accuracy = comparison_df.iloc[0]['Model']
best_model_by_accuracy, best_trainer_by_accuracy = trained_models[best_model_name_by_accuracy]

# Use accuracy-based selection
best_model_name = best_model_name_by_accuracy
best_model = best_model_by_accuracy
best_trainer = best_trainer_by_accuracy

print(f"\n{'='*60}")
print(f"🏆 Best Model (by Val Accuracy): {best_model_name}")
print(f"{'='*60}")
best_row = comparison_df[comparison_df['Model'] == best_model_name].iloc[0]
print(f"  Validation Loss: {best_row['Val Loss']:.4f}")
print(f"  Validation Accuracy: {best_row['Val Accuracy']:.2f}%")
print(f"  Train Accuracy: {best_row['Train Accuracy']:.2f}%")
print(f"  Generalization Gap: {best_row['Train Accuracy'] - best_row['Val Accuracy']:.2f}%")

# Detailed evaluation of best model
print(f"\n{'='*60}")
print(f"Detailed Evaluation: {best_model_name}")
print(f"{'='*60}")

# Make predictions
y_pred = best_trainer.predict(X_val)
y_pred_binary = (y_pred > 0.5).astype(int).flatten()

# Calculate metrics
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

precision = precision_score(y_val, y_pred_binary, zero_division=0)
recall = recall_score(y_val, y_pred_binary, zero_division=0)
f1 = f1_score(y_val, y_pred_binary, zero_division=0)

print(f"\nMetrics:")
print(f"  Precision: {precision:.4f}")
print(f"  Recall: {recall:.4f}")
print(f"  F1-Score: {f1:.4f}")

print(f"\nConfusion Matrix:")
cm = confusion_matrix(y_val, y_pred_binary)
print(f"                Predicted")
print(f"              ↓ (0)   ↑ (1)")
print(f"Actual ↓ (0)  {cm[0,0]:6d}  {cm[0,1]:6d}")
print(f"       ↑ (1)  {cm[1,0]:6d}  {cm[1,1]:6d}")

print(f"\nClassification Report:")
print(classification_report(y_val, y_pred_binary, target_names=['↓ Lower', '↑ Higher']))



Model Comparison

 Model  Val Loss  Val Accuracy  Train Accuracy  Class 0 Acc  Class 1 Acc   Balance
Hybrid  0.417458     54.573333       60.052000    44.489989    62.481229 82.008760
  LSTM  0.440989     53.980000       69.881333    32.933943    71.421971 61.511972
Conv1D  0.392557     45.466667       47.466667    92.606022     8.178353 15.572330
   GRU  0.770712     43.326667       51.368000    97.319880     3.765739  6.445859

🏆 Best Model (by Val Accuracy): Hybrid
  Validation Loss: 0.4175
  Validation Accuracy: 54.57%
  Train Accuracy: 60.05%
  Generalization Gap: 5.48%

Detailed Evaluation: Hybrid

Metrics:
  Precision: 0.6057
  Recall: 0.6248
  F1-Score: 0.6151

Confusion Matrix:
                Predicted
              ↓ (0)   ↑ (1)
Actual ↓ (0)    2822    3521
       ↑ (1)    3248    5409

Classification Report:
              precision    recall  f1-score   support

     ↓ Lower       0.46      0.44      0.45      6343
    ↑ Higher       0.61      0.62      0.62      8657

   

## 6. Save Models

Save all trained models and the best model for later use.


## 7. Summary

Training complete! The best model has been selected and saved. Next steps:
1. Use `04_evaluation.ipynb` for detailed evaluation and visualization
2. Use `05_submission.ipynb` to generate Kaggle predictions

In [32]:
# Save models - Only save best model to save disk space
model_dir = Path("../models")
model_dir.mkdir(exist_ok=True)

print(f"\nSaving models to {model_dir}...")

# Only save best model (saves disk space)
best_params = best_model.get_params()
with open(model_dir / "best_model_params.pkl", "wb") as f:
    pickle.dump(best_params, f)
print(f"✓ Saved best model ({best_model_name}) parameters")

# Save model comparison results (small file)
with open(model_dir / "model_comparison.pkl", "wb") as f:
    pickle.dump({
        'comparison': comparison_df.to_dict('records'),
        'best_model': best_model_name,
        'models_results': models_results,
        'sequence_length': SEQUENCE_LENGTH,
        'feature_cols': feature_cols
    }, f)
print(f"✓ Model comparison saved")

# Optionally save other models (commented out to save disk space)
# Uncomment if you need all models saved
# for model_name, (model, trainer) in trained_models.items():
#     if model_name != best_model_name:  # Skip best model (already saved)
#         params = model.get_params()
#         with open(model_dir / f"{model_name.lower()}_params.pkl", "wb") as f:
#             pickle.dump(params, f)
#         print(f"✓ Saved {model_name} parameters")

print(f"\n✓ Models saved!")
print(f"✓ Best model: {best_model_name}")



Saving models to ../models...
✓ Saved best model (Hybrid) parameters
✓ Model comparison saved

✓ Models saved!
✓ Best model: Hybrid
